In [41]:
from huggingface_hub import notebook_login

notebook_login()

In [49]:
from datasets import load_dataset

template = """<bos><start_of_turn>user
{human}<end_of_turn>
<start_of_turn>model
{gpt}<|eot_id|>"""

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [template.format(human=convo[0]["value"], gpt=convo[1]["value"]) for convo in convos]
    return { "text" : texts, }
pass

dataset_llm = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["train.jsonl"], split = "train")
dataset_llm = dataset_llm.map(formatting_prompts_func, batched = True)

In [55]:
print(dataset_llm[0]['text'])

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: A [Quello di fiori si riceve volentieri] [Vezzi visibili] RL [Dividono l'Argentina dal Cile] SE
Chiave di lettura: 8 9<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- A = A
- [Quello di fiori si riceve volentieri] = mazzo
- [Vezzi visibili] = nei
- R L = R L
- [Dividono l'Argentina dal Cile] = Ande
- S E = S E

Prima lettura: A mazzo nei RL Ande SE

Ora componiamo la soluzione seguendo la chiave risolutiva:
8 = Amazzone
9 = irlandese

Soluzione: Amazzone irlandese
<|eot_id|>


In [57]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig  # Note: This might be a typo. In your code, you later call LoraConfig.
from trl import GRPOConfig, GRPOTrainer

In [ ]:
# System prompt that instructs the model to use a specific XML format.
SYSTEM_PROMPT = """
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.


<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [ ]:
# XML chain-of-thought format template.
XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

# Function to extract the answer part from the XML response.
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

# Function to extract an answer if it is provided with a "####" delimiter.
def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()


In [46]:
dataset = load_dataset('gsarti/eureka-rebus', 'verbalized_rebus', split = "train")

In [47]:
dataset[0]

{'FRASE': "Non vi sono difficoltà a l'amore sincero",
 'PRIMALET': 'N O N - viso - nodi - F fico - L T A A L - amo RE - S in cero',
 'AUTORE': '-',
 'RIVISTA': 'RPC',
 'MESE': 0,
 'ANNO': 1869,
 'TIPO': None,
 'NOTE': None,
 'WORDS': 'viso nodi fico amo in cero',
 'LETTERS': 'N O N F L T A A L R E S',
 'FRASE_LEN': "3 2 4 10 1 1 ' 5 7",
 'FRASE_SEPARATED': "Non vi sono difficoltà a l ' amore sincero",
 'VERBALIZED_PRIMALET': "N O N [Volto, faccia] [Possono essere scorsoi] F [C'è anche quello d'India] L T A A L [Un' insidia subacquea] RE S [Alla moda... inglese] [Si accende in chiesa]",
 'VERBALIZED_PRIMALET_WITH_LEN': "N O N [Volto, faccia (4)] [Possono essere scorsoi (4)] F [C'è anche quello d'India (4)] L T A A L [Un' insidia subacquea (3)] RE S [Alla moda... inglese (2)] [Si accende in chiesa (4)]"}

In [54]:
prompt = f"{dataset[0]['VERBALIZED_PRIMALET']} \n" + f"{dataset[0]['FRASE_LEN']} \n\n"

# altro

Sia per le soluzioni delle definizioni che per le parole della soluzione finale, direi:
- Differenza di lunghezza con lunghezza giusta
- Edit distance tra predicted e correct words
- (se si riesce abbastanza rapidamente) semantic similarity tra predicted e correct word

In [7]:
import re 

def extract_solution(text):
    match = re.search(r"Soluzione:\s*(\w+\s\w+)", text)
    if match:
        solution = match.group(1)
    return solution

In [16]:
extract_solution(dataset[0]['text'])

'Amazzone irlandese'

- Proportion of correctly guessed words during definition resolution (i.e. `mazzo`, `nei`, ...)
- Proportion of correct words and letters in the generated first pass (i.e. `Prima lettura: A mazzo nei RL Ande SE`)
- Proportion of generated first passes matching the gold reference (`A-mazzo-nei-RL-ande-SE` - `amazzone-irlandese`)
- Proportion of generated solution words (`amazzone`, `irlandese`) matching the lengths specified by the solution key (`8`, `9`)
- 